In [ ]:
import json, pathlib, pandas as pd
import numpy as np

path = pathlib.Path("/home/neocortex/repos/LLaMEA/exp-07-17_081651-LLaMEA-o4-mini-2025-04-16-ELA/log.jsonl")      # 🔴 update if needed
records = [json.loads(line) for line in path.open()]

df = pd.json_normalize(records)
print(df.columns)

df = df[["fitness", "name", "code", "metadata.score1", "metadata.score2", "metadata.ela_features"]]
df = df.rename(columns={"metadata.score1": "score1", "metadata.score2": "score2", "metadata.ela_features": "ela"})
df.dropna(inplace=True)  # drop rows with NaN values

# -- unwrap [[...]] → [...]
df["ela"] = df["ela"].apply(lambda v: v[0] if isinstance(v, (list, tuple)) and len(v)==1 else v)
print(df["ela"])
# -- explode into separate numeric columns
ela_mat = np.vstack(df["ela"].values)
ela_cols = [f"ela_{i}" for i in range(ela_mat.shape[1])]
ela_df   = pd.DataFrame(ela_mat, columns=ela_cols, index=df.index)

df = pd.concat([df.drop(columns=["ela"]), ela_df], axis=1)

df.head()        # sanity-check


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Filter the best fitness values above 0.8 threshold
df_best = df[df["fitness"] > 0.8]

X_scaled = StandardScaler().fit_transform(df_best[ela_cols])
pc       = PCA(n_components=2).fit_transform(X_scaled)

fig, ax = plt.subplots(figsize=(5, 5))
sc      = ax.scatter(pc[:, 0], pc[:, 1], c=df_best["fitness"], cmap="viridis")
ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_title("ELA feature PCA (color = fitness)")
plt.colorbar(sc, label="fitness")
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
sc      = ax.scatter(df["score2"], df["score1"], c=df["fitness"], cmap="viridis")
ax.set_xlabel("score 2")
ax.set_ylabel("score 1")
ax.set_title("score1 vs score2 (color = fitness)")
plt.colorbar(sc, label="fitness")
plt.tight_layout()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_landscape(row, bounds=(-5, 5), res=200):
    """
    row  : one row of the dataframe (df.loc[idx] or df.iloc[idx])
    bounds: (low, high) tuple applied to both axes
    res   : grid resolution per axis
    """
    # Re-create the callable from the code string
    ns = {}
    exec(row["code"], ns)                      # defines class with same name as row["name"]
    f   = getattr(ns[row["name"]](dim=2), "f")      # instantiate & grab its .f method

    # Grid
    x = np.linspace(bounds[0], bounds[1], res)
    y = np.linspace(bounds[0], bounds[1], res)
    X, Y = np.meshgrid(x, y)
    pts   = np.stack([X.ravel(), Y.ravel()], axis=1)
    Z     = np.apply_along_axis(f, 1, pts).reshape(X.shape)   # robust fallback

    # Plot
    fig, ax = plt.subplots(figsize=(6, 5))
    cs = ax.contourf(X, Y, Z, levels=50, cmap="viridis")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(f"Landscape of {row['name']}")
    plt.colorbar(cs, ax=ax, label="f(x)")
    plt.tight_layout()
    return fig, ax

# Example usage:
top10 = df.nlargest(10, "fitness").reset_index(drop=True)
for idx in range(len(top10)):
    fig, ax = plot_landscape(top10.iloc[idx], bounds=(-5, 5))
    plt.show()  # Show each plot one by one


In [ ]:
# --- helper that draws on an existing axis instead of creating a new figure
def _plot_on_ax(row, ax, bounds=(-5, 5), res=150):
    ns = {}
    exec(row["code"], ns)
    f = getattr(ns[row["name"]](dim=2), "f")

    x = np.linspace(*bounds, res)
    y = np.linspace(*bounds, res)
    X, Y = np.meshgrid(x, y)
    Z = np.apply_along_axis(f, 1, np.stack([X.ravel(), Y.ravel()], axis=1)).reshape(X.shape)

    ax.contourf(X, Y, Z, levels=80, cmap="viridis")
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_title(f"score: {row['fitness']:.2g}", fontsize=7)

top64 = df.nlargest(64, "fitness").reset_index(drop=True)

rows, cols = 8, 8
fig, axes = plt.subplots(rows, cols, figsize=(14, 14), dpi=300)

for ax in axes.ravel():
    ax.axis("off")           # blank everything first

for ax, (_, row) in zip(axes.ravel(), top64.iterrows()):
    _plot_on_ax(row, ax, bounds=(-5, 5), res=200)

plt.tight_layout()